<a href="https://colab.research.google.com/github/shineloveyc/Doing_ML/blob/master/Preprocessing_Structured_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Categorical variable encoding

## Lable Encoding

In [0]:
#convert Pandas cat data for sklearn
from sklearn import preprocessing
import pandas as pd
import numpy as np

In [170]:
#create data fram
raw_data = {'patient':[1,2,3,3,2,3],
            'obs':[1,2,3,3,1,1],
            'score':['strong','weak','strong','strong','weak','weak']}

df = pd.DataFrame(raw_data, columns=['patient', 'obs', 'score'])
df

,patient,obs,score
0,1,1,strong
1,2,2,weak
2,3,3,strong
3,3,3,strong
4,2,1,weak
5,3,1,weak


In [114]:
#fit the label encoder
from sklearn.preprocessing import LabelEncoder

label_encode = LabelEncoder()

df['score'] = label_encode.fit_transform(df['score'])

df

,patient,obs,score
0,1,1,0
1,2,2,1
2,3,3,0
3,3,3,0
4,2,1,1
5,3,1,1


In [115]:
#list classed
list(label_encode.classes_)

['strong', 'weak']

In [116]:
#check inverse of encoding
list(label_encode.inverse_transform([1,0,1]))

['weak', 'strong', 'weak']

## One Hot Encoding

In [159]:
#using pd.get_dummies
df_dummy = pd.get_dummies(df)
df_dummy

,patient,obs,score_strong,score_weak
0,1,1,1,0
1,2,2,0,1
2,3,3,1,0
3,3,3,1,0
4,2,1,0,1
5,3,1,0,1


In [186]:
#using mutltlable encoder
from sklearn.preprocessing import MultiLabelBinarizer

# Create NumPy array
y = [('Texas', 'Florida'), 
    ('California', 'Alabama'), 
    ('Texas', 'Florida'), 
    ('Delware', 'Florida'), 
    ('Texas', 'Alabama')]

m_encode = MultiLabelBinarizer()

m_encode.fit_transform(y)

array([[0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]])

In [175]:
m_encode.classes_

array(['Alabama', 'California', 'Delware', 'Florida', 'Texas'],
      dtype=object)

In [181]:
#One Hot Encoding
from sklearn.preprocessing import OneHotEncoder

one_hot = OneHotEncoder()

one_hot.fit_transform(y)

one_hot.categories_

[array(['California', 'Delware', 'Texas'], dtype=object),
 array(['Alabama', 'Florida'], dtype=object)]

# Dealing Missing Value

In [0]:
#delete obs with missing value
#create a feature matrix
X = np.array([[1.1, 11.1],
              [2.2, 22.2],
              [3.3, 33.3],
              [4.4, 44.4],
              [np.nan, 55]])

In [118]:
#delet obs with missing value
X[~np.isnan(X).any(axis = 1)]

array([[ 1.1, 11.1],
       [ 2.2, 22.2],
       [ 3.3, 33.3],
       [ 4.4, 44.4]])

In [119]:
#delete missing values
X= np.array([[1,2],
             [6,3],
             [np.nan, 5]])
X[~np.isnan(X).any(axis = 1)]

array([[1., 2.],
       [6., 3.]])

In [120]:
#use pandas to drop missing value
df = pd.DataFrame(X, columns=['a','b'])

df.dropna()

,a,b
0,1.0,2.0
1,6.0,3.0


In [121]:
#impute missing values
X_df = pd.DataFrame(X, columns=['col_A', 'col_B'])
X_df

,col_A,col_B
0,1.0,2.0
1,6.0,3.0
2,NaN,5.0


In [122]:
from sklearn.impute import SimpleImputer

#fit imputer, replace the value to column mean
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

imputer_df = mean_imputer.fit_transform(X_df)
imputer_df

array([[1. , 2. ],
       [6. , 3. ],
       [3.5, 5. ]])

In [123]:
#impute class labels
#create data fram
df_data = {'patient':[1,2,3,3,2,3],
            'obs':[1,2,3,3,1,1],
            'score':[np.nan,'weak','strong','strong','weak','weak']}

df = pd.DataFrame(df_data, columns=['patient', 'obs', 'score'])
df

,patient,obs,score
0,1,1,NaN
1,2,2,weak
2,3,3,strong
3,3,3,strong
4,2,1,weak
5,3,1,weak


In [124]:
freq_impute = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

df_new = freq_impute.fit_transform(df)
df_new

array([[1, 1, 'weak'],
       [2, 2, 'weak'],
       [3, 3, 'strong'],
       [3, 3, 'strong'],
       [2, 1, 'weak'],
       [3, 1, 'weak']], dtype=object)

In [149]:
#impute with KNN
#create trainning data
X = np.array([[0, 2.10, 1.45], 
              [1, 1.18, 1.33], 
              [0, 1.22, 1.27],
              [1, -0.21, -1.19]])
#creat feature matrix with missing valuee
X_with_nan = np.array([[np.nan, 0.87, 1.31],
                       [np.nan, -0.67, -0.22]])

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(3, weights='distance')

model = knn.fit(X[:,1:], X[:, 0])

imputed_values = model.predict(X_with_nan[:,1:])

X_with_imputed = np.hstack((imputed_values.reshape(-1, 1), X_with_nan[:, 1:]))

X_with_imputed 

#join two dfs
np.vstack((X_with_imputed,X))

array([[ 0.  ,  0.87,  1.31],
       [ 1.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

# Dealing Outliers

In [0]:
#delete outliers
from sklearn.covariance import EllipticEnvelope
from sklearn.datasets import make_blobs

In [126]:
#create simulated data
X, _ = make_blobs(n_samples=10, 
                  n_features = 2,
                  centers =1 ,
                  random_state = 1)

#replace the first obs with extreme outliers
X[0, 0] = 10000
X[0, 1] =10000
X

array([[ 1.00000000e+04,  1.00000000e+04],
       [-2.76017908e+00,  5.55121358e+00],
       [-1.61734616e+00,  4.98930508e+00],
       [-5.25790464e-01,  3.30659860e+00],
       [ 8.52518583e-02,  3.64528297e+00],
       [-7.94152277e-01,  2.10495117e+00],
       [-1.34052081e+00,  4.15711949e+00],
       [-1.98197711e+00,  4.02243551e+00],
       [-2.18773166e+00,  3.33352125e+00],
       [-1.97451969e-01,  2.34634916e+00]])

 EllipticEnvelope assumes the data is normally distributed and based on that assumption “draws” an ellipse around the data, classifying any observation inside the ellipse as an inlier (labeled as 1) and any observation outside the ellipse as an outlier (labeled as -1). A major limitation of this approach is the need to specify a contamination parameter which is the proportion of observations that are outliers, a value that we don’t know.



In [127]:
#create detector
outlier_dector = EllipticEnvelope(contamination=0.1)

#fit the detector
outlier_dector.fit(X)

outlier_dector.predict(X)

array([-1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [0]:
#Handeling Outliers
#create data frame
house = pd.DataFrame()

house['Price'] = [4000, 5000, 8000, 400000]
house['Bathrooms'] = [2.5, 1, 3, 116]
house['Sqrt_Feet'] = ['1500', '2000', '2000', '48000']

In [129]:
#option one, drop outlier
house.describe()

#after observe the outlier, drop obs
house[house['Bathrooms']<20]

,Price,Bathrooms,Sqrt_Feet
0,4000,2.5,1500
1,5000,1.0,2000
2,8000,3.0,2000


In [130]:
#option two, mark outlier
house['outlier'] = np.where(house['Bathrooms']<20, 0, 1)
house

,Price,Bathrooms,Sqrt_Feet,outlier
0,4000,2.5,1500,0
1,5000,1.0,2000,0
2,8000,3.0,2000,0
3,400000,116.0,48000,1


In [131]:
#option 3, rescale
house['log of price'] = [np.log(x) for x in house['Price']]
house

,Price,Bathrooms,Sqrt_Feet,outlier,log of price
0,4000,2.5,1500,0,8.294050
1,5000,1.0,2000,0,8.517193
2,8000,3.0,2000,0,8.987197
3,400000,116.0,48000,1,12.899220


# Discretize Features

In [0]:
#Discretize Features
from sklearn.preprocessing import Binarizer

#create data
age = np.array([[13],
                [44],
                [55],
                [20],
               [90]])

In [133]:
#option one, binarize feature, use 18 as watershed
binarizer = Binarizer(18)

binarizer.fit_transform(age)

array([[0],
       [1],
       [1],
       [1],
       [1]])

In [134]:
#option 2, break up feature into bins
np.digitize(age, bins = [20, 30, 64])

array([[0],
       [2],
       [2],
       [1],
       [3]])

In [135]:
#encoding Ordinal categorical features
df = pd.DataFrame({'Score':['Low',
                            'High',
                            'Low',
                            'Medium']})
df

,Score
0,Low
1,High
2,Low
3,Medium


In [136]:
#create a map
scale_map = {'Low': 0,
             'Medium':1,
             'High':2}

#map scale to features
df['Score'] = df['Score'].replace(scale_map)
df

,Score
0,0
1,2
2,0
3,1


In downsampling, we randomly sample without replacement from the majority class (i.e. the class with more observations) to create a new subset of observation equal in size to the minority class

# Dealing Imbalanced Data

In [0]:
#Handel imbalance data
from sklearn.datasets import load_iris

In [0]:
iris = load_iris()

X = iris.data

y= iris.target

In [139]:
#make Iris dataste imbalance

X = X[40:, :]
y = y[40:]

#change y to binary target vector
y = np.where((y==0), 0, 1)

y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [140]:
#downsampling the majority class 1 to match minority class

#Indicies of each class's obs
i_class0 = np.where(y==0)[0]
i_class1 = np.where(y==1)[0]

#number of obs in each class
print('class 0 is', len(i_class0))
print('class 1 is', len(i_class1))

#for every obs of 0, randomly sample from class 1 without replacement
#Generates a random sample from a given 1-D array
i_class1_downsampled = np.random.choice(i_class1, size = len(i_class0), replace = False)

#join new sample with original set==size is 20
np.hstack((y[i_class0], y[i_class1_downsampled]))

class 0 is 10
class 1 is 100


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [141]:
#upsampling, for every obs in class 1, randomly sample from class 0 with replacement)
i_class0_upsampled = np.random.choice(i_class0, size = len(i_class1), replace = True)
#join two samples, but need to take out old class 0 samples
final = np.concatenate((y[i_class1], y[i_class0_upsampled]))
final.size

200

# Data Normalization

In [153]:
#http://mathworld.wolfram.com/L2-Norm.html
from sklearn.preprocessing import Normalizer
#create feature matrix

X = np.array([[.5, .5],
              [1.1, 3.4],
              [1.5, 2.62],
              [1.63, 34.4],
              [10.9, 3.3]])

#initiate normilizer
normalizer = Normalizer(norm = 'l2')
#every element divide its length( which is sqrt(a**2 + b**2))==>l2 norm
#l1 norm is absoluted value of the vector length
normalizer.transform(X)

array([[0.70710678, 0.70710678],
       [0.30782029, 0.95144452],
       [0.4968522 , 0.86783518],
       [0.04733062, 0.99887928],
       [0.95709822, 0.28976368]])

In [188]:
#rescale using Min- Max
from sklearn.preprocessing import MinMaxScaler

MinMaxScaler(feature_range=(0,1)).fit_transform(X)

array([[0.        , 0.        ],
       [0.05769231, 0.08554572],
       [0.09615385, 0.06253687],
       [0.10865385, 1.        ],
       [1.        , 0.08259587]])

In [190]:
#standardize features

from sklearn.preprocessing import StandardScaler

StandardScaler().fit_transform(X)

array([[-0.67215216, -0.65082607],
       [-0.51857589, -0.42462813],
       [-0.4161917 , -0.48546758],
       [-0.38291684,  1.99334984],
       [ 1.9898366 , -0.43242806]])

# Apply Data Preprocessing to iris data

In [0]:
iris = load_iris()

X = iris.data

y= iris.target

In [0]:
#split data for cross validation
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)

In [215]:
y_train[:5]

array([2, 2, 1, 0, 0])

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

In [0]:
iris_pipe = Pipeline(steps = [('scaler', StandardScaler()),('SVM', SVC())])

In [0]:
#initiate parameters for svm
parameters = {'SVM__C':[0.001, 0.1, 10, 100], 'SVM__gamma':[0.1, 0.01]}

In [0]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(iris_pipe, param_grid=parameters, cv = 3)

In [220]:
#train the data with grid search CV
grid.fit(X_train,y_train)

print('score = %3.2f' %(grid.score(X_test,y_test)))

score = 0.96
